## Installation

In [ ]:
!pip install -U tqdm
!pip install -U adapter-transformers
!pip install -U datasets
!pip install -U optuna

     |████████████████████████████████| 76 kB 3.4 MB/s 
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1
     |████████████████████████████████| 2.5 MB 9.1 MB/s 
     |████████████████████████████████| 43 kB 1.7 MB/s 
     |████████████████████████████████| 895 kB 52.2 MB/s 
     |████████████████████████████████| 3.3 MB 56.6 MB/s 
     |████████████████████████████████| 542 kB 9.1 MB/s 
     |████████████████████████████████| 243 kB 19.4 MB/s 
     |████████████████████████████████| 118 kB 20.0 MB/s 
     |████████████████████████████████| 301 kB 9.9 MB/s 
     |████████████████████████████████| 80 kB 8.6 MB/s 
     |████████████████████████████████| 164 kB 16.5 MB/s 
     |████████████████████████████████| 75 kB 5.0 MB/s 
     |████████████████████████████████| 49 kB 6.9 MB/s 
     |████████████████████████████████| 111 kB 20.7 MB/s 
     |████████████████████████████████| 141 kB 17.8 MB

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Wed Jul 28 05:56:32 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

To enable a high-RAM runtime, select the Runtime > "Change runtime type"
menu, and then select High-RAM in the Runtime shape dropdown. Then, 
re-execute this cell.


## Dataset


In [ ]:
import datasets
from datasets import load_dataset, load_metric
from sklearn import metrics
import torch
import numpy as np
import transformers
from transformers import TrainingArguments, Trainer, EvalPrediction

print(datasets.__version__)
print(torch.__version__)

#datasets.list_datasets()
#dataset.num_rows

1.10.2
1.9.0+cu102


In [ ]:
dataset = load_dataset('glue', 'cola')
metric = load_metric('glue', 'cola')
dataset

Downloading:   0%|          | 0.00/7.78k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.47k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/377k [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


Downloading:   0%|          | 0.00/1.86k [00:00<?, ?B/s]

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 8551
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1043
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1063
    })
})

In [ ]:
[max([s['sentence'].count(' ') for s in dataset[d]]) for d in dataset]

[41, 28, 25]

In [ ]:
from transformers import RobertaTokenizer, AutoTokenizer, AutoModelWithHeads, RobertaConfig, RobertaModelWithHeads, AutoConfig

model_name = "roberta-base"
#model_name = "bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_name)

def encode_batch(batch):
  """Encodes a batch of input data using the model tokenizer."""
  return tokenizer(batch["sentence"], max_length=128, truncation=True, padding="max_length")

# Encode the input data
dataset = dataset.map(encode_batch, batched=True, num_proc=2)
# The transformers model expects the target class column to be named "labels"
dataset.rename_column_("label", "labels")
# Transform to pytorch tensors and only output the required columns
dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: rename_column_ is deprecated and will be removed in the next major version of datasets. Use DatasetDict.rename_column instead.
  from ipykernel import kernelapp as app


In [ ]:
#np.unique(np.array([d['labels'].numpy() for d in dataset['test']]))

## Optimization

In [ ]:
# from transformers import RobertaConfig, RobertaModelWithHeads

# config = RobertaConfig.from_pretrained(
#     "roberta-base",
#     num_labels=2,
# )
# model = RobertaModelWithHeads.from_pretrained(
#     "roberta-base",
#     config=config,
# )

In [ ]:
# config = AutoConfig.from_pretrained(model_name, num_labels=2)
# model = AutoModelWithHeads.from_pretrained(model_name, config=config)

# adapter_config = transformers.PfeifferConfig(reduction_factor=32)
# model.add_adapter("cola", config=adapter_config)
# model.add_classification_head("cola", num_labels=2)
# model.train_adapter("cola")
# model.load_adapter("lingaccept/cola@ukp", config='houlsby')
# model.set_active_adapters("cola")

In [ ]:
#model.get_adapter('cola')

In [ ]:
def compute_metrics(p: EvalPrediction):
    
   preds = np.argmax(p.predictions, axis=1)
   acc = (preds == p.label_ids).mean()
   mat = metrics.matthews_corrcoef(p.label_ids, preds)
   f1 = metrics.f1_score(p.label_ids, preds)
   #corr = np.corrcoef(p.label_ids)

   #return {"acc": acc, "matthews_corr": mat, "F1":f1}
   return metric.compute(predictions=preds, references= p.label_ids)
   #return {"matthews_corr": mat}

In [ ]:
def model_init():
    
    config = AutoConfig.from_pretrained(model_name, num_labels=2)
    model = AutoModelWithHeads.from_pretrained(model_name, config=config)
    
    adapter_config = transformers.PfeifferConfig(reduction_factor=128)
    model.add_adapter("cola", config=adapter_config)
    model.add_classification_head("cola", num_labels=2)
    model.train_adapter("cola")
    return model

def hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-5, 1e-3, log=True),
        "num_train_epochs": trial.suggest_int("num_train_epochs", 3, 20),
        #"seed": trial.suggest_int("seed", 1, 40),
        #"per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [4, 8, 16, 32, 64]),
    }

# def hp_space(trial):
#     from ray import tune

#     return {
#         "learning_rate": tune.loguniform(1e-5, 1e-3),
#         "num_train_epochs": tune.choice(range(3, 15)),
#         #"seed": tune.choice(range(1, 41)),
#         "per_device_train_batch_size": tune.choice([4, 8, 16, 32, 64]),
#     }

training_args = TrainingArguments(
    #learning_rate=1e-4,
    #num_train_epochs=6,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    #evaluate_during_training=True, 
    evaluation_strategy = 'epoch',
    #eval_steps=500,
    logging_steps=500,
    output_dir="./training_output",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
)

#model = model_init()
trainer = Trainer(
    #model=model,
    args=training_args,
    #tokenizer=tokenizer,
    model_init = model_init,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    compute_metrics=compute_metrics,
)

loading configuration file https://huggingface.co/roberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b
Model config RobertaConfig {
  "adapters": {
    "adapters": {},
    "config_map": {}
  },
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "2.1.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file https:

In [ ]:
# del model
# del trainer

# import gc
# gc.collect()
# torch.cuda.empty_cache()

In [ ]:
best_trial = trainer.hyperparameter_search(
    hp_space = hp_space,
    direction = "maximize",
    n_trials = 20,
    #backend = "optuna",
    #n_jobs = 4,
    #n_samples = 10,
)

[I 2021-07-22 18:32:12,851] A new study created in memory with name: no-name-3e405697-a0cd-4f91-8f41-dc045d8f0c11
Trial:
loading configuration file https://huggingface.co/roberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b
Model config RobertaConfig {
  "adapters": {
    "adapters": {},
    "config_map": {}
  },
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transfor

Epoch,Training Loss,Validation Loss,Matthews Correlation
1,No log,0.502003,0.436016
2,0.502600,0.424575,0.550860
3,0.502600,0.492637,0.482941
4,0.407000,0.483069,0.542267
5,0.407000,0.435804,0.552078
6,0.372000,0.434557,0.557276
7,0.372000,0.445198,0.563783


***** Running Evaluation *****
  Num examples = 1043
  Batch size = 32
Saving model checkpoint to ./training_output/run-0/checkpoint-500
Configuration saved in ./training_output/run-0/checkpoint-500/cola/adapter_config.json
Module weights saved in ./training_output/run-0/checkpoint-500/cola/pytorch_adapter.bin
Configuration saved in ./training_output/run-0/checkpoint-500/cola/head_config.json
Module weights saved in ./training_output/run-0/checkpoint-500/cola/pytorch_model_head.bin
Configuration saved in ./training_output/run-0/checkpoint-500/cola/head_config.json
Module weights saved in ./training_output/run-0/checkpoint-500/cola/pytorch_model_head.bin
***** Running Evaluation *****
  Num examples = 1043
  Batch size = 32
***** Running Evaluation *****
  Num examples = 1043
  Batch size = 32
Saving model checkpoint to ./training_output/run-0/checkpoint-1000
Configuration saved in ./training_output/run-0/checkpoint-1000/cola/adapter_config.json
Module weights saved in ./training_output

Epoch,Training Loss,Validation Loss,Matthews Correlation
1,No log,0.610655,0.000000
2,0.607400,0.576608,0.000000
3,0.607400,0.501245,0.388716
4,0.500300,0.526842,0.392702
5,0.500300,0.498360,0.452422
6,0.450800,0.483901,0.461206
7,0.450800,0.494376,0.466549
8,0.429000,0.465774,0.498168
9,0.429000,0.486785,0.472390
10,0.424400,0.494389,0.472390


***** Running Evaluation *****
  Num examples = 1043
  Batch size = 32
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning:

invalid value encountered in double_scalars

Saving model checkpoint to ./training_output/run-1/checkpoint-500
Configuration saved in ./training_output/run-1/checkpoint-500/cola/adapter_config.json
Module weights saved in ./training_output/run-1/checkpoint-500/cola/pytorch_adapter.bin
Configuration saved in ./training_output/run-1/checkpoint-500/cola/head_config.json
Module weights saved in ./training_output/run-1/checkpoint-500/cola/pytorch_model_head.bin
Configuration saved in ./training_output/run-1/checkpoint-500/cola/head_config.json
Module weights saved in ./training_output/run-1/checkpoint-500/cola/pytorch_model_head.bin
***** Running Evaluation *****
  Num examples = 1043
  Batch size = 32
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning:

invalid value encountered in d

...

***** Running Evaluation *****
  Num examples = 1043
  Batch size = 32
[I 2021-07-22 22:02:34,865] Trial 17 pruned. 
Trial:
loading configuration file https://huggingface.co/roberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b
Model config RobertaConfig {
  "adapters": {
    "adapters": {},
    "config_map": {}
  },
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "trans

Epoch,Training Loss,Validation Loss,Matthews Correlation
1,No log,0.562345,0.188310


***** Running Evaluation *****
  Num examples = 1043
  Batch size = 32
[I 2021-07-22 22:03:55,817] Trial 18 pruned. 
Trial:
loading configuration file https://huggingface.co/roberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b
Model config RobertaConfig {
  "adapters": {
    "adapters": {},
    "config_map": {}
  },
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "trans

Epoch,Training Loss,Validation Loss,Matthews Correlation
1,No log,0.614481,0.000000


***** Running Evaluation *****
  Num examples = 1043
  Batch size = 32
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning:

invalid value encountered in double_scalars

[I 2021-07-22 22:05:16,746] Trial 19 pruned. 


In [ ]:
best_trial

BestRun(run_id='14', objective=0.6286336357917458, hyperparameters={'learning_rate': 0.000995778875486788, 'num_train_epochs': 19})

## Main

In [ ]:
import functools

def compute_metrics(p: EvalPrediction):
    
   preds = np.argmax(p.predictions, axis=1)
   acc = (preds == p.label_ids).mean()
   mat = metrics.matthews_corrcoef(p.label_ids, preds)
   f1 = metrics.f1_score(p.label_ids, preds)

   return metric.compute(predictions=preds, references= p.label_ids)

training_args = TrainingArguments(
    learning_rate=0.0001,
    num_train_epochs=30,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    evaluation_strategy = 'epoch',
    logging_strategy = 'epoch',
    #eval_steps=500,
    #logging_steps=500,
    output_dir="./training_output",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
    metric_for_best_model = 'eval_matthews_correlation',
    load_best_model_at_end = True,
)


create_trainer = functools.partial(Trainer, args=training_args, 
                                   train_dataset=dataset["train"],
                                   eval_dataset=dataset["validation"],
                                   compute_metrics=compute_metrics,)

In [ ]:
for red in [2, 16, 64, 256]:
  print('\n\n************************************************************')
  print('Reduction_factor = ',red)
  config = RobertaConfig.from_pretrained(model_name, num_labels=2)
  model = RobertaModelWithHeads.from_pretrained(model_name, config=config)
  adapter_config = transformers.PfeifferConfig(reduction_factor=red)
  model.add_adapter("cola", config=adapter_config)
  model.add_classification_head("cola", num_labels=2)
  model.train_adapter("cola")

  trainer = Trainer(
      model=model,
      args=training_args,
      #tokenizer=tokenizer,
      #model_init = model_init,
      train_dataset=dataset["train"],
      eval_dataset=dataset["validation"],
      compute_metrics=compute_metrics,
  )

  trainer.train()
  display(trainer.evaluate())

************************************************************
Reduction_factor =  2


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModelWithHeads: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModelWithHeads were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infere

Epoch,Training Loss,Validation Loss,Matthews Correlation
1,0.509000,0.589734,0.458993
2,0.390500,0.548295,0.541655
3,0.336100,0.493057,0.577918
4,0.279500,0.431293,0.631787
5,0.237200,0.519448,0.595815
6,0.196400,0.620128,0.603317
7,0.164400,0.644509,0.586313
8,0.145900,0.656671,0.593281
9,0.128600,0.667013,0.596174
10,0.119800,0.645461,0.598573


***** Running Evaluation *****
  Num examples = 1043
  Batch size = 32
Saving model checkpoint to ./training_output/checkpoint-268
Configuration saved in ./training_output/checkpoint-268/cola/adapter_config.json
Module weights saved in ./training_output/checkpoint-268/cola/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-268/cola/head_config.json
Module weights saved in ./training_output/checkpoint-268/cola/pytorch_model_head.bin
Configuration saved in ./training_output/checkpoint-268/cola/head_config.json
Module weights saved in ./training_output/checkpoint-268/cola/pytorch_model_head.bin
tokenizer config file saved in ./training_output/checkpoint-268/tokenizer_config.json
Special tokens file saved in ./training_output/checkpoint-268/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 1043
  Batch size = 32
Saving model checkpoint to ./training_output/checkpoint-536
Configuration saved in ./training_output/checkpoint-536/cola/adapter_config.j

************************************************************
Reduction_factor =  16


...

***** Running Evaluation *****
  Num examples = 1043
  Batch size = 32
Saving model checkpoint to ./training_output/checkpoint-268
Configuration saved in ./training_output/checkpoint-268/cola/adapter_config.json
Module weights saved in ./training_output/checkpoint-268/cola/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-268/cola/head_config.json
Module weights saved in ./training_output/checkpoint-268/cola/pytorch_model_head.bin
Configuration saved in ./training_output/checkpoint-268/cola/head_config.json
Module weights saved in ./training_output/checkpoint-268/cola/pytorch_model_head.bin
tokenizer config file saved in ./training_output/checkpoint-268/tokenizer_config.json
Special tokens file saved in ./training_output/checkpoint-268/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 1043
  Batch size = 32
Saving model checkpoint to ./training_output/checkpoint-536
Configuration saved in ./training_output/checkpoint-536/cola/adapter_config.j

************************************************************
Reduction_factor =  256


loading configuration file https://huggingface.co/roberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b
Model config RobertaConfig {
  "adapters": {
    "adapters": {},
    "config_map": {}
  },
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "2.1.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file https:

Epoch,Training Loss,Validation Loss,Matthews Correlation
1,0.605000,0.567503,0.000000
2,0.495000,0.470330,0.488446
3,0.443900,0.497147,0.469217
4,0.430200,0.467598,0.493854
5,0.409800,0.464449,0.520756
6,0.405700,0.473181,0.509989
7,0.390500,0.435582,0.555443
8,0.390100,0.460472,0.539126
9,0.375400,0.451865,0.547487
10,0.372700,0.439051,0.555809


***** Running Evaluation *****
  Num examples = 1043
  Batch size = 32
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
Saving model checkpoint to ./training_output/checkpoint-268
Configuration saved in ./training_output/checkpoint-268/cola/adapter_config.json
Module weights saved in ./training_output/checkpoint-268/cola/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-268/cola/head_config.json
Module weights saved in ./training_output/checkpoint-268/cola/pytorch_model_head.bin
Configuration saved in ./training_output/checkpoint-268/cola/head_config.json
Module weights saved in ./training_output/checkpoint-268/cola/pytorch_model_head.bin
tokenizer config file saved in ./training_output/checkpoint-268/tokenizer_config.json
Special tokens file saved in ./training_output/checkpoint-268/special_tokens_map.json
***** Running Evaluat

## Ablation

**Ablation 1: Lower layers**

In [ ]:
config = RobertaConfig.from_pretrained(model_name, num_labels=2)
model = RobertaModelWithHeads.from_pretrained(model_name, config=config)

adapter_config = transformers.PfeifferConfig(reduction_factor={"0":256, "1":256, "2":256, "default": 2})
model.add_adapter("cola", config=adapter_config)
model.add_classification_head("cola", num_labels=2)
model.train_adapter("cola")

trainer = create_trainer(model)
trainer.train()
trainer.evaluate()

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModelWithHeads: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModelWithHeads were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infere

Epoch,Training Loss,Validation Loss,Matthews Correlation
1,0.540500,0.610001,0.444365
2,0.428600,0.532620,0.506820
3,0.370800,0.522263,0.555285
4,0.319800,0.424646,0.600984
5,0.276400,0.477334,0.572799
6,0.240000,0.560733,0.562455
7,0.203400,0.542599,0.590659
8,0.177100,0.599135,0.583169
9,0.155600,0.567605,0.609780
10,0.142900,0.776762,0.568142


***** Running Evaluation *****
  Num examples = 1043
  Batch size = 32
Saving model checkpoint to ./training_output/checkpoint-268
Configuration saved in ./training_output/checkpoint-268/cola/adapter_config.json
Module weights saved in ./training_output/checkpoint-268/cola/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-268/cola/head_config.json
Module weights saved in ./training_output/checkpoint-268/cola/pytorch_model_head.bin
Configuration saved in ./training_output/checkpoint-268/cola/head_config.json
Module weights saved in ./training_output/checkpoint-268/cola/pytorch_model_head.bin
***** Running Evaluation *****
  Num examples = 1043
  Batch size = 32
Saving model checkpoint to ./training_output/checkpoint-536
Configuration saved in ./training_output/checkpoint-536/cola/adapter_config.json
Module weights saved in ./training_output/checkpoint-536/cola/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-536/cola/head_config.json
Module w

{'epoch': 30.0,
 'eval_loss': 1.0831646919250488,
 'eval_matthews_correlation': 0.6262696480582318,
 'eval_runtime': 4.8539,
 'eval_samples_per_second': 214.879,
 'eval_steps_per_second': 6.799}

**Ablation 2:**

In [ ]:
config = RobertaConfig.from_pretrained(model_name, num_labels=2)
model = RobertaModelWithHeads.from_pretrained(model_name, config=config)

adapter_config = transformers.PfeifferConfig(reduction_factor={"9":256, "10":256, "11":256, "default": 2})
model.add_adapter("cola", config=adapter_config)
model.add_classification_head("cola", num_labels=2)
model.train_adapter("cola")

trainer = create_trainer(model)
trainer.train()
trainer.evaluate()

loading configuration file https://huggingface.co/roberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b
Model config RobertaConfig {
  "adapters": {
    "adapters": {},
    "config_map": {}
  },
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "2.1.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file https:

Epoch,Training Loss,Validation Loss,Matthews Correlation
1,0.541000,0.623500,0.452454
2,0.421900,0.576339,0.499515
3,0.362300,0.506114,0.531171
4,0.308900,0.432602,0.585349
5,0.262300,0.515187,0.558246
6,0.228100,0.581513,0.557240
7,0.188700,0.590177,0.554658
8,0.167700,0.627619,0.568736
9,0.141100,0.697744,0.577811
10,0.137200,0.704279,0.588298


***** Running Evaluation *****
  Num examples = 1043
  Batch size = 32
Saving model checkpoint to ./training_output/checkpoint-268
Configuration saved in ./training_output/checkpoint-268/cola/adapter_config.json
Module weights saved in ./training_output/checkpoint-268/cola/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-268/cola/head_config.json
Module weights saved in ./training_output/checkpoint-268/cola/pytorch_model_head.bin
Configuration saved in ./training_output/checkpoint-268/cola/head_config.json
Module weights saved in ./training_output/checkpoint-268/cola/pytorch_model_head.bin
***** Running Evaluation *****
  Num examples = 1043
  Batch size = 32
Saving model checkpoint to ./training_output/checkpoint-536
Configuration saved in ./training_output/checkpoint-536/cola/adapter_config.json
Module weights saved in ./training_output/checkpoint-536/cola/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-536/cola/head_config.json
Module w

{'epoch': 30.0,
 'eval_loss': 1.1269817352294922,
 'eval_matthews_correlation': 0.6240786802650855,
 'eval_runtime': 4.8677,
 'eval_samples_per_second': 214.271,
 'eval_steps_per_second': 6.779}

**Ablation 3:**

In [ ]:
config = RobertaConfig.from_pretrained(model_name, num_labels=2)
model = RobertaModelWithHeads.from_pretrained(model_name, config=config)

adapter_config = transformers.PfeifferConfig(reduction_factor={"8":256, "7":256, "6":256, "default": 2})
model.add_adapter("cola", config=adapter_config)
model.add_classification_head("cola", num_labels=2)
model.train_adapter("cola")

trainer = create_trainer(model)
trainer.train()
trainer.evaluate()

loading configuration file https://huggingface.co/roberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b
Model config RobertaConfig {
  "adapters": {
    "adapters": {},
    "config_map": {}
  },
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "2.1.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file https:

Epoch,Training Loss,Validation Loss,Matthews Correlation
1,0.538600,0.658154,0.395854
2,0.406700,0.571452,0.500296
3,0.355900,0.532409,0.547514
4,0.298500,0.450677,0.585866
5,0.258400,0.497861,0.578205
6,0.229800,0.605704,0.582240
7,0.190300,0.487135,0.585573
8,0.163000,0.714060,0.565754
9,0.140300,0.814567,0.562623
10,0.143500,0.622671,0.585536


***** Running Evaluation *****
  Num examples = 1043
  Batch size = 32
Saving model checkpoint to ./training_output/checkpoint-268
Configuration saved in ./training_output/checkpoint-268/cola/adapter_config.json
Module weights saved in ./training_output/checkpoint-268/cola/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-268/cola/head_config.json
Module weights saved in ./training_output/checkpoint-268/cola/pytorch_model_head.bin
Configuration saved in ./training_output/checkpoint-268/cola/head_config.json
Module weights saved in ./training_output/checkpoint-268/cola/pytorch_model_head.bin
***** Running Evaluation *****
  Num examples = 1043
  Batch size = 32
Saving model checkpoint to ./training_output/checkpoint-536
Configuration saved in ./training_output/checkpoint-536/cola/adapter_config.json
Module weights saved in ./training_output/checkpoint-536/cola/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-536/cola/head_config.json
Module w

{'epoch': 30.0,
 'eval_loss': 0.5466495752334595,
 'eval_matthews_correlation': 0.6064593797294897,
 'eval_runtime': 4.8619,
 'eval_samples_per_second': 214.527,
 'eval_steps_per_second': 6.788}

In [ ]:
config = RobertaConfig.from_pretrained(model_name, num_labels=2)
model = RobertaModelWithHeads.from_pretrained(model_name, config=config)

adapter_config = transformers.PfeifferConfig(reduction_factor={"3":256, "4":256, "5":256, "default": 2})
model.add_adapter("cola", config=adapter_config)
model.add_classification_head("cola", num_labels=2)
model.train_adapter("cola")

trainer = create_trainer(model)
trainer.train()
trainer.evaluate()

loading configuration file https://huggingface.co/roberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b
Model config RobertaConfig {
  "adapters": {
    "adapters": {},
    "config_map": {}
  },
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "2.1.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file https:

Epoch,Training Loss,Validation Loss,Matthews Correlation
1,0.536400,0.606333,0.412377
2,0.410700,0.611808,0.490589
3,0.354100,0.549541,0.514231
4,0.305000,0.461543,0.565160
5,0.258200,0.532151,0.568142
6,0.220600,0.582078,0.567677
7,0.186300,0.595365,0.575972
8,0.167200,0.611440,0.611091
9,0.141200,0.652654,0.608229
10,0.132600,0.664657,0.585706


***** Running Evaluation *****
  Num examples = 1043
  Batch size = 32
Saving model checkpoint to ./training_output/checkpoint-268
Configuration saved in ./training_output/checkpoint-268/cola/adapter_config.json
Module weights saved in ./training_output/checkpoint-268/cola/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-268/cola/head_config.json
Module weights saved in ./training_output/checkpoint-268/cola/pytorch_model_head.bin
Configuration saved in ./training_output/checkpoint-268/cola/head_config.json
Module weights saved in ./training_output/checkpoint-268/cola/pytorch_model_head.bin
***** Running Evaluation *****
  Num examples = 1043
  Batch size = 32
Saving model checkpoint to ./training_output/checkpoint-536
Configuration saved in ./training_output/checkpoint-536/cola/adapter_config.json
Module weights saved in ./training_output/checkpoint-536/cola/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-536/cola/head_config.json
Module w

{'epoch': 30.0,
 'eval_loss': 1.1057647466659546,
 'eval_matthews_correlation': 0.6258269726983982,
 'eval_runtime': 4.8842,
 'eval_samples_per_second': 213.546,
 'eval_steps_per_second': 6.756}

## Base

In [ ]:
config = RobertaConfig.from_pretrained(model_name, num_labels=2)
model = RobertaModelWithHeads.from_pretrained(model_name, config=config)

#adapter_config = transformers.PfeifferConfig(reduction_factor=16)
#model.add_adapter("cola", config=adapter_config)
model.add_classification_head("cola", num_labels=2)
#model.train_adapter("cola")
model


loading configuration file https://huggingface.co/roberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b
Model config RobertaConfig {
  "adapters": {
    "adapters": {},
    "config_map": {}
  },
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "2.1.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file https:

RobertaModelWithHeads(
  (roberta): RobertaModel(
    (invertible_adapters): ModuleDict()
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bia

In [ ]:
for name, param in model.named_parameters():
  #print(name)
  if 'cola' not in name:
    param.requires_grad = False
  else:
    print(name)

heads.cola.1.weight
heads.cola.1.bias
heads.cola.4.weight
heads.cola.4.bias


In [ ]:
trainer = create_trainer(model)
trainer.train()
trainer.evaluate()

***** Running training *****
  Num examples = 8551
  Num Epochs = 30
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 8040


Epoch,Training Loss,Validation Loss,Matthews Correlation
1,0.610400,0.611670,0.000000
2,0.600700,0.602055,0.000000
3,0.593400,0.595618,0.000000
4,0.587100,0.588946,0.000000
5,0.582600,0.582544,0.000000
6,0.576500,0.579365,0.000000
7,0.571500,0.582285,0.000000
8,0.574400,0.570849,0.065589
9,0.568400,0.572860,0.065589
10,0.568200,0.563024,0.073807


***** Running Evaluation *****
  Num examples = 1043
  Batch size = 32
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
Saving model checkpoint to ./training_output/checkpoint-268
Configuration saved in ./training_output/checkpoint-268/config.json
Model weights saved in ./training_output/checkpoint-268/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1043
  Batch size = 32
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
Saving model checkpoint to ./training_output/checkpoint-536
Configuration saved in ./training_output/checkpoint-536/config.json
Model weights saved in ./training_output/checkpoint-536/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1043
  Batch size = 32
/usr/loca

{'epoch': 30.0,
 'eval_loss': 0.542631208896637,
 'eval_matthews_correlation': 0.2279360616503532,
 'eval_runtime': 4.5045,
 'eval_samples_per_second': 231.548,
 'eval_steps_per_second': 7.326}

## Adapter Fusion

In [ ]:
from transformers.adapters.composition import Fuse

config = RobertaConfig.from_pretrained(model_name, num_labels=2)
model = RobertaModelWithHeads.from_pretrained(model_name, config=config)

adapter_config = transformers.PfeifferConfig()
model.load_adapter("nli/multinli@ukp", load_as="multinli", with_head=False)
model.load_adapter("sts/qqp@ukp", with_head=False)
model.load_adapter("nli/qnli@ukp", with_head=False)
model.add_adapter_fusion(Fuse("multinli", "qqp", "qnli"))
model.set_active_adapters(Fuse("multinli", "qqp", "qnli"))
#model.add_adapter("cola", config=adapter_config)

model.add_classification_head("cola", num_labels=2)

adapter_setup = Fuse("multinli", "qqp", "qnli")
model.train_adapter_fusion(adapter_setup)
#model.train_adapter("cola")

trainer = create_trainer(model)

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModelWithHeads: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModelWithHeads were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infere

Downloading:   0%|          | 0.00/540 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.33M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.53M [00:00<?, ?B/s]

In [ ]:
trainer.train()
trainer.evaluate()

***** Running training *****
  Num examples = 8551
  Num Epochs = 30
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 8040


Epoch,Training Loss,Validation Loss,Matthews Correlation
1,0.511800,0.689740,0.358391
2,0.450700,0.483386,0.502412
3,0.431800,0.515783,0.472382
4,0.417700,0.425558,0.528505
5,0.409100,0.418846,0.547644
6,0.411700,0.462777,0.507428
7,0.405700,0.414194,0.554397
8,0.409400,0.443834,0.531198
9,0.402600,0.398549,0.571213
10,0.388600,0.446336,0.494827


***** Running Evaluation *****
  Num examples = 1043
  Batch size = 32
Saving model checkpoint to ./training_output/checkpoint-268
Configuration saved in ./training_output/checkpoint-268/multinli/adapter_config.json
Module weights saved in ./training_output/checkpoint-268/multinli/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-268/qqp/adapter_config.json
Module weights saved in ./training_output/checkpoint-268/qqp/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-268/qnli/adapter_config.json
Module weights saved in ./training_output/checkpoint-268/qnli/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-268/multinli,qqp,qnli/adapter_fusion_config.json
Module weights saved in ./training_output/checkpoint-268/multinli,qqp,qnli/pytorch_model_adapter_fusion.bin
***** Running Evaluation *****
  Num examples = 1043
  Batch size = 32
Saving model checkpoint to ./training_output/checkpoint-536
Configuration saved in ./training_

{'epoch': 30.0,
 'eval_loss': 0.4254721701145172,
 'eval_matthews_correlation': 0.5992421641912468,
 'eval_runtime': 7.6531,
 'eval_samples_per_second': 136.285,
 'eval_steps_per_second': 4.312}

**Add CoLA to Fusion**

In [ ]:
config = RobertaConfig.from_pretrained(model_name, num_labels=2)
model = RobertaModelWithHeads.from_pretrained(model_name, config=config)

adapter_config = transformers.PfeifferConfig(reduction_factor=16)
model.add_adapter("cola", config=adapter_config)
model.add_classification_head("cola", num_labels=2)
model.train_adapter("cola")

trainer = create_trainer(model)
trainer.train()
trainer.evaluate()

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModelWithHeads: ['lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModelWithHeads were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infere

Epoch,Training Loss,Validation Loss,Matthews Correlation
1,0.557400,0.490924,0.477905
2,0.440300,0.594486,0.418455
3,0.399200,0.544883,0.515291
4,0.361900,0.413942,0.575641
5,0.335600,0.478634,0.542309
6,0.318100,0.530317,0.544411
7,0.291700,0.443412,0.578479
8,0.271200,0.548328,0.563013
9,0.245500,0.499306,0.585536
10,0.242200,0.497786,0.573254


***** Running Evaluation *****
  Num examples = 1043
  Batch size = 32
Saving model checkpoint to ./training_output/checkpoint-268
Configuration saved in ./training_output/checkpoint-268/cola/adapter_config.json
Module weights saved in ./training_output/checkpoint-268/cola/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-268/cola/head_config.json
Module weights saved in ./training_output/checkpoint-268/cola/pytorch_model_head.bin
Configuration saved in ./training_output/checkpoint-268/cola/head_config.json
Module weights saved in ./training_output/checkpoint-268/cola/pytorch_model_head.bin
***** Running Evaluation *****
  Num examples = 1043
  Batch size = 32
Saving model checkpoint to ./training_output/checkpoint-536
Configuration saved in ./training_output/checkpoint-536/cola/adapter_config.json
Module weights saved in ./training_output/checkpoint-536/cola/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-536/cola/head_config.json
Module w

{'epoch': 30.0,
 'eval_loss': 0.5027017593383789,
 'eval_matthews_correlation': 0.6209103735772308,
 'eval_runtime': 4.6536,
 'eval_samples_per_second': 224.129,
 'eval_steps_per_second': 7.091}

In [ ]:
model.save_adapter("./cola_roberta", "cola")

Configuration saved in ./cola_roberta/adapter_config.json
Module weights saved in ./cola_roberta/pytorch_adapter.bin
Configuration saved in ./cola_roberta/head_config.json
Module weights saved in ./cola_roberta/pytorch_model_head.bin


**Fusion test**

In [ ]:
from transformers.adapters.composition import Fuse

config = RobertaConfig.from_pretrained(model_name, num_labels=2)
model = RobertaModelWithHeads.from_pretrained(model_name, config=config)

model.load_adapter("./cola_roberta", load_as="cola", with_head=False)

model.add_adapter_fusion(Fuse("cola"))
model.set_active_adapters(Fuse("cola"))

model.add_classification_head("cola", num_labels=2)

adapter_setup = Fuse("cola")
model.train_adapter_fusion(adapter_setup)

trainer = create_trainer(model)

loading configuration file https://huggingface.co/roberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b
Model config RobertaConfig {
  "adapters": {
    "adapters": {},
    "config_map": {}
  },
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "2.1.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file https:

In [ ]:
trainer.train()
trainer.evaluate()

***** Running training *****
  Num examples = 8551
  Num Epochs = 30
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 8040


Epoch,Training Loss,Validation Loss,Matthews Correlation
1,0.228200,0.612490,0.563188
2,0.197300,0.646221,0.571799
3,0.189200,0.762658,0.561876
4,0.188100,0.644808,0.600954
5,0.182300,0.647215,0.585962
6,0.190500,0.611082,0.600954
7,0.171900,0.591960,0.585442
8,0.177300,0.629348,0.598107
9,0.173700,0.528204,0.613388
10,0.176900,0.492163,0.636678


***** Running Evaluation *****
  Num examples = 1043
  Batch size = 32
Saving model checkpoint to ./training_output/checkpoint-268
Configuration saved in ./training_output/checkpoint-268/cola/adapter_config.json
Module weights saved in ./training_output/checkpoint-268/cola/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-268/cola/head_config.json
Module weights saved in ./training_output/checkpoint-268/cola/pytorch_model_head.bin
Configuration saved in ./training_output/checkpoint-268/cola/head_config.json
Module weights saved in ./training_output/checkpoint-268/cola/pytorch_model_head.bin
Configuration saved in ./training_output/checkpoint-268/cola/adapter_fusion_config.json
Module weights saved in ./training_output/checkpoint-268/cola/pytorch_model_adapter_fusion.bin
***** Running Evaluation *****
  Num examples = 1043
  Batch size = 32
Saving model checkpoint to ./training_output/checkpoint-536
Configuration saved in ./training_output/checkpoint-536/cola/adapt

{'epoch': 30.0,
 'eval_loss': 0.47894206643104553,
 'eval_matthews_correlation': 0.6404826984976293,
 'eval_runtime': 6.068,
 'eval_samples_per_second': 171.885,
 'eval_steps_per_second': 5.438}

In [ ]:
model.save_adapter_fusion("./cola_roberta_fusion_cola", "cola")

Configuration saved in ./cola_roberta_fusion_cola/adapter_fusion_config.json
Module weights saved in ./cola_roberta_fusion_cola/pytorch_model_adapter_fusion.bin


**Fusion with inference tasks**

In [ ]:
config = RobertaConfig.from_pretrained(model_name, num_labels=2)
model = RobertaModelWithHeads.from_pretrained(model_name, config=config)

model.load_adapter("nli/multinli@ukp", load_as="multinli", with_head=False)
model.load_adapter("nli/qnli@ukp", with_head=False)
model.load_adapter("./cola_roberta", load_as="cola", with_head=False)

model.add_adapter_fusion(Fuse("cola","multinli", "qnli"))
model.set_active_adapters(Fuse("cola","multinli", "qnli"))

model.add_classification_head("cola", num_labels=2)

adapter_setup = Fuse("cola", "multinli", "qnli")
model.train_adapter_fusion(adapter_setup)

trainer = create_trainer(model)

loading configuration file https://huggingface.co/roberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b
Model config RobertaConfig {
  "adapters": {
    "adapters": {},
    "config_map": {}
  },
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "2.1.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file https:

Downloading:   0%|          | 0.00/3.32M [00:00<?, ?B/s]

storing https://public.ukp.informatik.tu-darmstadt.de/AdapterHub/text_task/mnli/roberta-base/pfeiffer/multinli_relu_16.zip in cache at ~/.cache/torch/adapters/a4b20be37c13e64cda7f46716bdda616abb4b396b90a61c7044244d99f5e275d.76b733dc2b30f0f82e23270b627d8237231cd7a8a90c43e2aaa6c9b6881ff1ba
creating metadata file for ~/.cache/torch/adapters/a4b20be37c13e64cda7f46716bdda616abb4b396b90a61c7044244d99f5e275d.76b733dc2b30f0f82e23270b627d8237231cd7a8a90c43e2aaa6c9b6881ff1ba
Loading module configuration from ~/.cache/torch/adapters/a4b20be37c13e64cda7f46716bdda616abb4b396b90a61c7044244d99f5e275d-76b733dc2b30f0f82e23270b627d8237231cd7a8a90c43e2aaa6c9b6881ff1ba-extracted/adapter_config.json
Adding adapter 'multinli'.
Loading module weights from ~/.cache/torch/adapters/a4b20be37c13e64cda7f46716bdda616abb4b396b90a61c7044244d99f5e275d-76b733dc2b30f0f82e23270b627d8237231cd7a8a90c43e2aaa6c9b6881ff1ba-extracted/pytorch_adapter.bin
No exactly matching adapter config found for this specifier, falling back

Downloading:   0%|          | 0.00/5.53M [00:00<?, ?B/s]

storing https://public.ukp.informatik.tu-darmstadt.de/AdapterHub/text_task/qnli/roberta-base/pfeiffer/roberta-base_qnli_pfeiffer.zip in cache at ~/.cache/torch/adapters/7fea166c43ea2b6a0b91575e85a2af9ac75f505fd65c4fce9db0009316afe1fa.da33c659efdfaac763f2bc9077cfb1559a0134e2c429bff45ee185634510d03c
creating metadata file for ~/.cache/torch/adapters/7fea166c43ea2b6a0b91575e85a2af9ac75f505fd65c4fce9db0009316afe1fa.da33c659efdfaac763f2bc9077cfb1559a0134e2c429bff45ee185634510d03c
Loading module configuration from ~/.cache/torch/adapters/7fea166c43ea2b6a0b91575e85a2af9ac75f505fd65c4fce9db0009316afe1fa-da33c659efdfaac763f2bc9077cfb1559a0134e2c429bff45ee185634510d03c-extracted/adapter_config.json
Adding adapter 'qnli'.
Loading module weights from ~/.cache/torch/adapters/7fea166c43ea2b6a0b91575e85a2af9ac75f505fd65c4fce9db0009316afe1fa-da33c659efdfaac763f2bc9077cfb1559a0134e2c429bff45ee185634510d03c-extracted/pytorch_adapter.bin
Some module weights could not be found in loaded weights file: robe

In [ ]:
trainer.train()
trainer.evaluate()

***** Running training *****
  Num examples = 8551
  Num Epochs = 30
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 8040


Epoch,Training Loss,Validation Loss,Matthews Correlation
1,0.254400,0.651408,0.580345
2,0.219300,0.605762,0.590554
3,0.190400,0.603600,0.572700
4,0.192300,0.589491,0.605659
5,0.185500,0.693442,0.586123
6,0.193800,0.602339,0.598246
7,0.174300,0.578914,0.595816
8,0.177400,0.612154,0.611204
9,0.179300,0.528716,0.621954
10,0.187000,0.489218,0.637977


***** Running Evaluation *****
  Num examples = 1043
  Batch size = 32
Saving model checkpoint to ./training_output/checkpoint-268
Configuration saved in ./training_output/checkpoint-268/multinli/adapter_config.json
Module weights saved in ./training_output/checkpoint-268/multinli/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-268/qnli/adapter_config.json
Module weights saved in ./training_output/checkpoint-268/qnli/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-268/cola/adapter_config.json
Module weights saved in ./training_output/checkpoint-268/cola/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-268/cola/head_config.json
Module weights saved in ./training_output/checkpoint-268/cola/pytorch_model_head.bin
Configuration saved in ./training_output/checkpoint-268/cola/head_config.json
Module weights saved in ./training_output/checkpoint-268/cola/pytorch_model_head.bin
Configuration saved in ./training_output/check

{'epoch': 30.0,
 'eval_loss': 0.46081843972206116,
 'eval_matthews_correlation': 0.641021499856239,
 'eval_runtime': 7.6684,
 'eval_samples_per_second': 136.012,
 'eval_steps_per_second': 4.303}

In [ ]:
model.save_adapter_fusion("./cola_roberta_fusion_nli", "cola,multinli,qnli")

Configuration saved in ./cola_roberta_fusion_nli/adapter_fusion_config.json
Module weights saved in ./cola_roberta_fusion_nli/pytorch_model_adapter_fusion.bin


In [ ]:
config = RobertaConfig.from_pretrained(model_name, num_labels=2)
model = RobertaModelWithHeads.from_pretrained(model_name, config=config)

model.load_adapter("nli/multinli@ukp", load_as="multinli", with_head=False)
model.load_adapter("nli/qnli@ukp", with_head=False)

model.add_adapter_fusion(Fuse("multinli", "qnli"))
model.set_active_adapters(Fuse("multinli", "qnli"))

model.add_classification_head("cola", num_labels=2)

adapter_setup = Fuse("multinli", "qnli")
model.train_adapter_fusion(adapter_setup)

trainer = create_trainer(model)

loading configuration file https://huggingface.co/roberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b
Model config RobertaConfig {
  "adapters": {
    "adapters": {},
    "config_map": {}
  },
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "2.1.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file https:

In [ ]:
trainer.train()
trainer.evaluate()

***** Running training *****
  Num examples = 8551
  Num Epochs = 30
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 8040


Epoch,Training Loss,Validation Loss,Matthews Correlation
1,0.520400,0.579859,0.451022
2,0.449500,0.639743,0.475289
3,0.427300,0.488156,0.528526
4,0.422100,0.405054,0.576304
5,0.409600,0.436160,0.539044
6,0.413000,0.521209,0.500543
7,0.400000,0.433837,0.562781
8,0.396800,0.447642,0.554700
9,0.396700,0.402070,0.566035
10,0.381400,0.437701,0.547487


***** Running Evaluation *****
  Num examples = 1043
  Batch size = 32
Saving model checkpoint to ./training_output/checkpoint-268
Configuration saved in ./training_output/checkpoint-268/multinli/adapter_config.json
Module weights saved in ./training_output/checkpoint-268/multinli/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-268/qnli/adapter_config.json
Module weights saved in ./training_output/checkpoint-268/qnli/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-268/multinli,qnli/adapter_fusion_config.json
Module weights saved in ./training_output/checkpoint-268/multinli,qnli/pytorch_model_adapter_fusion.bin
***** Running Evaluation *****
  Num examples = 1043
  Batch size = 32
Saving model checkpoint to ./training_output/checkpoint-536
Configuration saved in ./training_output/checkpoint-536/multinli/adapter_config.json
Module weights saved in ./training_output/checkpoint-536/multinli/pytorch_adapter.bin
Configuration saved in ./trainin

{'epoch': 30.0,
 'eval_loss': 0.4141436219215393,
 'eval_matthews_correlation': 0.5803084401024122,
 'eval_runtime': 6.9159,
 'eval_samples_per_second': 150.813,
 'eval_steps_per_second': 4.772}

**Fusion with semantic tasks**

In [ ]:
config = RobertaConfig.from_pretrained(model_name, num_labels=2)
model = RobertaModelWithHeads.from_pretrained(model_name, config=config)

model.load_adapter("sts/mrpc@ukp", with_head=False)
model.load_adapter("sts/qqp@ukp", with_head=False)
model.load_adapter("./cola_roberta", load_as="cola", with_head=False)

model.add_adapter_fusion(Fuse("cola", "mrpc", "qqp"))
model.set_active_adapters(Fuse("cola", "mrpc", "qqp"))

model.add_classification_head("cola", num_labels=2)

adapter_setup = Fuse("cola", "mrpc", "qqp")
model.train_adapter_fusion(adapter_setup)

trainer = create_trainer(model)

loading configuration file https://huggingface.co/roberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b
Model config RobertaConfig {
  "adapters": {
    "adapters": {},
    "config_map": {}
  },
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "2.1.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file https:

In [ ]:
trainer.train()
trainer.evaluate()

***** Running training *****
  Num examples = 8551
  Num Epochs = 30
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 8040


Epoch,Training Loss,Validation Loss,Matthews Correlation
1,0.237700,0.664197,0.540596
2,0.204200,0.570328,0.591175
3,0.188800,0.676242,0.552542
4,0.191900,0.673046,0.580530
5,0.189700,0.647023,0.596211
6,0.191300,0.618352,0.590554
7,0.173500,0.556364,0.578332
8,0.180000,0.608843,0.600879
9,0.176400,0.535393,0.613867
10,0.180300,0.470215,0.627978


***** Running Evaluation *****
  Num examples = 1043
  Batch size = 32
Saving model checkpoint to ./training_output/checkpoint-268
Configuration saved in ./training_output/checkpoint-268/mrpc/adapter_config.json
Module weights saved in ./training_output/checkpoint-268/mrpc/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-268/qqp/adapter_config.json
Module weights saved in ./training_output/checkpoint-268/qqp/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-268/cola/adapter_config.json
Module weights saved in ./training_output/checkpoint-268/cola/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-268/cola/head_config.json
Module weights saved in ./training_output/checkpoint-268/cola/pytorch_model_head.bin
Configuration saved in ./training_output/checkpoint-268/cola/head_config.json
Module weights saved in ./training_output/checkpoint-268/cola/pytorch_model_head.bin
Configuration saved in ./training_output/checkpoint-268/

{'epoch': 30.0,
 'eval_loss': 0.5191537141799927,
 'eval_matthews_correlation': 0.6317871775012913,
 'eval_runtime': 7.6569,
 'eval_samples_per_second': 136.217,
 'eval_steps_per_second': 4.31}

In [ ]:
model.save_adapter_fusion("./cola_roberta_fusion_semantic", "cola,mrpc,qqp")

Configuration saved in ./cola_roberta_fusion_semantic/adapter_fusion_config.json
Module weights saved in ./cola_roberta_fusion_semantic/pytorch_model_adapter_fusion.bin


In [ ]:
config = RobertaConfig.from_pretrained(model_name, num_labels=2)
model = RobertaModelWithHeads.from_pretrained(model_name, config=config)

model.load_adapter("sts/mrpc@ukp", with_head=False)
model.load_adapter("sts/qqp@ukp", with_head=False)

model.add_adapter_fusion(Fuse("mrpc", "qqp"))
model.set_active_adapters(Fuse("mrpc", "qqp"))

model.add_classification_head("cola", num_labels=2)

adapter_setup = Fuse("mrpc", "qqp")
model.train_adapter_fusion(adapter_setup)

trainer = create_trainer(model)

loading configuration file https://huggingface.co/roberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b
Model config RobertaConfig {
  "adapters": {
    "adapters": {},
    "config_map": {}
  },
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "2.1.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file https:

In [ ]:
trainer.train()
trainer.evaluate()

***** Running training *****
  Num examples = 8551
  Num Epochs = 30
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 8040


Epoch,Training Loss,Validation Loss,Matthews Correlation
1,0.531500,0.556513,0.422134
2,0.453900,0.568118,0.411751
3,0.435400,0.570202,0.419500
4,0.429200,0.443029,0.520951
5,0.414200,0.429749,0.546908
6,0.413900,0.561336,0.465116
7,0.412700,0.467058,0.493805
8,0.405700,0.444413,0.529668
9,0.407900,0.427719,0.536115
10,0.407200,0.420221,0.545606


***** Running Evaluation *****
  Num examples = 1043
  Batch size = 32
Saving model checkpoint to ./training_output/checkpoint-268
Configuration saved in ./training_output/checkpoint-268/mrpc/adapter_config.json
Module weights saved in ./training_output/checkpoint-268/mrpc/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-268/qqp/adapter_config.json
Module weights saved in ./training_output/checkpoint-268/qqp/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-268/mrpc,qqp/adapter_fusion_config.json
Module weights saved in ./training_output/checkpoint-268/mrpc,qqp/pytorch_model_adapter_fusion.bin
***** Running Evaluation *****
  Num examples = 1043
  Batch size = 32
Saving model checkpoint to ./training_output/checkpoint-536
Configuration saved in ./training_output/checkpoint-536/mrpc/adapter_config.json
Module weights saved in ./training_output/checkpoint-536/mrpc/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-536/qqp/

{'epoch': 30.0,
 'eval_loss': 0.43799349665641785,
 'eval_matthews_correlation': 0.5805514135255713,
 'eval_runtime': 6.8729,
 'eval_samples_per_second': 151.756,
 'eval_steps_per_second': 4.801}

**Adapter Fusion with sentiment tasks**

In [ ]:
config = RobertaConfig.from_pretrained(model_name, num_labels=2)
model = RobertaModelWithHeads.from_pretrained(model_name, config=config)

model.load_adapter("sentiment/sst-2@ukp", with_head=False)
model.load_adapter("sentiment/imdb@ukp", with_head=False)
model.load_adapter("./cola_roberta", load_as="cola", with_head=False)

model.add_adapter_fusion(Fuse("cola", "sst_glue", "imdb"))
model.set_active_adapters(Fuse("cola", "sst_glue", "imdb"))

model.add_classification_head("cola", num_labels=2)

adapter_setup = Fuse("cola", "sst_glue", "imdb")
model.train_adapter_fusion(adapter_setup)

trainer = create_trainer(model)

loading configuration file https://huggingface.co/roberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b
Model config RobertaConfig {
  "adapters": {
    "adapters": {},
    "config_map": {}
  },
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "2.1.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file https:

In [ ]:
trainer.train()
trainer.evaluate()

***** Running training *****
  Num examples = 8551
  Num Epochs = 30
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 8040


Epoch,Training Loss,Validation Loss,Matthews Correlation
1,0.256400,0.699107,0.536329
2,0.200500,0.598530,0.593405
3,0.186400,0.731538,0.547355
4,0.197500,0.511483,0.614834
5,0.186000,0.652771,0.575972
6,0.189600,0.623795,0.582996
7,0.175400,0.553577,0.563384
8,0.182800,0.612714,0.593131
9,0.185400,0.632474,0.595658
10,0.182200,0.471716,0.632695


***** Running Evaluation *****
  Num examples = 1043
  Batch size = 32
Saving model checkpoint to ./training_output/checkpoint-268
Configuration saved in ./training_output/checkpoint-268/sst_glue/adapter_config.json
Module weights saved in ./training_output/checkpoint-268/sst_glue/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-268/imdb/adapter_config.json
Module weights saved in ./training_output/checkpoint-268/imdb/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-268/cola/adapter_config.json
Module weights saved in ./training_output/checkpoint-268/cola/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-268/cola/head_config.json
Module weights saved in ./training_output/checkpoint-268/cola/pytorch_model_head.bin
Configuration saved in ./training_output/checkpoint-268/cola/head_config.json
Module weights saved in ./training_output/checkpoint-268/cola/pytorch_model_head.bin
Configuration saved in ./training_output/check

{'epoch': 30.0,
 'eval_loss': 0.45563897490501404,
 'eval_matthews_correlation': 0.6355969174067906,
 'eval_runtime': 7.6861,
 'eval_samples_per_second': 135.699,
 'eval_steps_per_second': 4.293}

In [ ]:
config = RobertaConfig.from_pretrained(model_name, num_labels=2)
model = RobertaModelWithHeads.from_pretrained(model_name, config=config)

model.load_adapter("sentiment/sst-2@ukp", with_head=False)
model.load_adapter("sentiment/imdb@ukp", with_head=False)
#model.load_adapter("./cola_roberta", load_as="cola", with_head=False)

model.add_adapter_fusion(Fuse("sst_glue", "imdb"))
model.set_active_adapters(Fuse("sst_glue", "imdb"))

model.add_classification_head("cola", num_labels=2)

adapter_setup = Fuse("sst_glue", "imdb")
model.train_adapter_fusion(adapter_setup)

trainer = create_trainer(model)

loading configuration file https://huggingface.co/roberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b
Model config RobertaConfig {
  "adapters": {
    "adapters": {},
    "config_map": {}
  },
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "2.1.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file https:

In [ ]:
trainer.train()
trainer.evaluate()

***** Running training *****
  Num examples = 8551
  Num Epochs = 30
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 8040


Epoch,Training Loss,Validation Loss,Matthews Correlation
1,0.515400,0.670735,0.336241
2,0.455500,0.496362,0.455784
3,0.442600,0.492880,0.491774
4,0.449900,0.459701,0.482939
5,0.424300,0.505807,0.477457
6,0.433600,0.579533,0.429346
7,0.422300,0.459839,0.499220
8,0.415600,0.486993,0.502074
9,0.411900,0.456301,0.518299
10,0.415200,0.529505,0.466442


***** Running Evaluation *****
  Num examples = 1043
  Batch size = 32
Saving model checkpoint to ./training_output/checkpoint-268
Configuration saved in ./training_output/checkpoint-268/sst_glue/adapter_config.json
Module weights saved in ./training_output/checkpoint-268/sst_glue/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-268/imdb/adapter_config.json
Module weights saved in ./training_output/checkpoint-268/imdb/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-268/sst_glue,imdb/adapter_fusion_config.json
Module weights saved in ./training_output/checkpoint-268/sst_glue,imdb/pytorch_model_adapter_fusion.bin
***** Running Evaluation *****
  Num examples = 1043
  Batch size = 32
Saving model checkpoint to ./training_output/checkpoint-536
Configuration saved in ./training_output/checkpoint-536/sst_glue/adapter_config.json
Module weights saved in ./training_output/checkpoint-536/sst_glue/pytorch_adapter.bin
Configuration saved in ./trainin

{'epoch': 30.0,
 'eval_loss': 0.4794031083583832,
 'eval_matthews_correlation': 0.5675517176765141,
 'eval_runtime': 6.8737,
 'eval_samples_per_second': 151.738,
 'eval_steps_per_second': 4.801}

**Combine six adapters**

In [ ]:
config = RobertaConfig.from_pretrained(model_name, num_labels=2)
model = RobertaModelWithHeads.from_pretrained(model_name, config=config)

model.load_adapter("sentiment/sst-2@ukp", with_head=False)
model.load_adapter("sentiment/imdb@ukp", with_head=False)
model.load_adapter("nli/multinli@ukp", load_as="multinli", with_head=False)
model.load_adapter("nli/qnli@ukp", with_head=False)
model.load_adapter("sts/mrpc@ukp", with_head=False)
model.load_adapter("sts/qqp@ukp", with_head=False)
model.load_adapter("./cola_roberta", load_as="cola", with_head=False)

model.add_adapter_fusion(Fuse("cola", "multinli", "qnli", "mrpc", "qqp", "sst_glue", "imdb"))
model.set_active_adapters(Fuse("cola", "multinli", "qnli", "mrpc", "qqp", "sst_glue", "imdb"))
model.add_classification_head("cola", num_labels=2)

adapter_setup = Fuse("cola","multinli", "qnli", "mrpc", "qqp", "sst_glue", "imdb")
model.train_adapter_fusion(adapter_setup)

trainer = create_trainer(model)

loading configuration file https://huggingface.co/roberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b
Model config RobertaConfig {
  "adapters": {
    "adapters": {},
    "config_map": {}
  },
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "2.1.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file https:

In [ ]:
trainer.train()
trainer.evaluate()

***** Running training *****
  Num examples = 8551
  Num Epochs = 30
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 8040


Epoch,Training Loss,Validation Loss,Matthews Correlation
1,0.257200,0.548067,0.581056
2,0.210900,0.620654,0.575804
3,0.219200,0.658861,0.565428
4,0.212200,0.555936,0.594786
5,0.209400,0.681526,0.570552
6,0.210200,0.539292,0.577818
7,0.188200,0.560354,0.552549
8,0.196100,0.611308,0.580397
9,0.197600,0.542446,0.596883
10,0.194900,0.506646,0.605746


***** Running Evaluation *****
  Num examples = 1043
  Batch size = 32
Saving model checkpoint to ./training_output/checkpoint-268
Configuration saved in ./training_output/checkpoint-268/sst_glue/adapter_config.json
Module weights saved in ./training_output/checkpoint-268/sst_glue/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-268/imdb/adapter_config.json
Module weights saved in ./training_output/checkpoint-268/imdb/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-268/multinli/adapter_config.json
Module weights saved in ./training_output/checkpoint-268/multinli/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-268/qnli/adapter_config.json
Module weights saved in ./training_output/checkpoint-268/qnli/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-268/mrpc/adapter_config.json
Module weights saved in ./training_output/checkpoint-268/mrpc/pytorch_adapter.bin
Configuration saved in ./training_outp

{'epoch': 30.0,
 'eval_loss': 0.44468289613723755,
 'eval_matthews_correlation': 0.639085890464288,
 'eval_runtime': 10.4244,
 'eval_samples_per_second': 100.053,
 'eval_steps_per_second': 3.166}

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
config = RobertaConfig.from_pretrained(model_name, num_labels=2)
model = RobertaModelWithHeads.from_pretrained(model_name, config=config)

model.load_adapter("sentiment/sst-2@ukp", with_head=False)
model.load_adapter("sentiment/imdb@ukp", with_head=False)
model.load_adapter("nli/multinli@ukp", load_as="multinli", with_head=False)
model.load_adapter("nli/qnli@ukp", with_head=False)
model.load_adapter("sts/mrpc@ukp", with_head=False)
model.load_adapter("sts/qqp@ukp", with_head=False)
#model.load_adapter("./cola_roberta", load_as="cola", with_head=False)

model.add_adapter_fusion(Fuse("multinli", "qnli", "mrpc", "qqp", "sst_glue", "imdb"))
model.set_active_adapters(Fuse("multinli", "qnli", "mrpc", "qqp", "sst_glue", "imdb"))

model.add_adapter_fusion(Fuse("multinli", "qnli", "mrpc", "qqp", "sst_glue", "imdb"))
model.set_active_adapters(Fuse("multinli", "qnli", "mrpc", "qqp", "sst_glue", "imdb"))

model.add_classification_head("cola", num_labels=2)

adapter_setup = Fuse("multinli", "qnli", "mrpc", "qqp", "sst_glue", "imdb")
model.train_adapter_fusion(adapter_setup)

trainer = create_trainer(model)

loading configuration file https://huggingface.co/roberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b
Model config RobertaConfig {
  "adapters": {
    "adapters": {},
    "config_map": {}
  },
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "2.1.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file https:

In [ ]:
trainer.train()
trainer.evaluate()

***** Running training *****
  Num examples = 8551
  Num Epochs = 30
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 8040


Epoch,Training Loss,Validation Loss,Matthews Correlation
1,0.513600,0.502008,0.477767
2,0.453400,0.554195,0.476880
3,0.432100,0.547520,0.494884
4,0.427700,0.414923,0.534266
5,0.412100,0.437899,0.520292
6,0.410000,0.471613,0.507304
7,0.399800,0.451095,0.529875
8,0.397700,0.469208,0.526405
9,0.397200,0.399254,0.535758
10,0.390100,0.487529,0.544754


***** Running Evaluation *****
  Num examples = 1043
  Batch size = 32
Saving model checkpoint to ./training_output/checkpoint-268
Configuration saved in ./training_output/checkpoint-268/sst_glue/adapter_config.json
Module weights saved in ./training_output/checkpoint-268/sst_glue/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-268/imdb/adapter_config.json
Module weights saved in ./training_output/checkpoint-268/imdb/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-268/multinli/adapter_config.json
Module weights saved in ./training_output/checkpoint-268/multinli/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-268/qnli/adapter_config.json
Module weights saved in ./training_output/checkpoint-268/qnli/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-268/mrpc/adapter_config.json
Module weights saved in ./training_output/checkpoint-268/mrpc/pytorch_adapter.bin
Configuration saved in ./training_outp

{'epoch': 30.0,
 'eval_loss': 0.41732341051101685,
 'eval_matthews_correlation': 0.5856159039288926,
 'eval_runtime': 9.7584,
 'eval_samples_per_second': 106.882,
 'eval_steps_per_second': 3.382}